<a href="https://colab.research.google.com/github/am-alarcon/iaPrimerParcial/blob/main/Parcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install optuna

In [27]:
import pandas as pd
import numpy as np
import tensorflow as tf
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error

In [28]:

SEED=1234


In [29]:
# Cargar el conjunto de datos
datos = pd.read_csv("Temp_Asu20092021.csv")
datos.head()

,Unnamed: 0,Fecha,Temperatura,Humedad
0,0,2009-01-01 00:00:00,26.749483,69.609451
1,1,2009-01-01 03:00:00,25.956114,78.225166
2,2,2009-01-01 06:00:00,23.707357,93.551756
3,3,2009-01-01 09:00:00,23.726823,93.394817
4,4,2009-01-01 12:00:00,29.606379,72.155839


In [11]:
# Convertir la columna 'Fecha' al tipo datetime y establecerla como índice
datos['Fecha'] = pd.to_datetime(datos['Fecha'])
datos.set_index('Fecha', inplace=True)
datos.head(9)


,Unnamed: 0,Temperatura,Humedad
Fecha,,,
2009-01-01 00:00:00,0,26.749483,69.609451
2009-01-01 03:00:00,1,25.956114,78.225166
2009-01-01 06:00:00,2,23.707357,93.551756
2009-01-01 09:00:00,3,23.726823,93.394817
2009-01-01 12:00:00,4,29.606379,72.155839


In [12]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 37984 entries, 2009-01-01 00:00:00 to 2021-12-31 21:00:00
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   37984 non-null  int64  
 1   Temperatura  37984 non-null  float64
 2   Humedad      37984 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 1.2 MB


In [25]:
datos.tail(9)

,Temperatura,Humedad
Fecha,,
2021-12-30 21:00:00,37.961710,25.495340
2021-12-31 00:00:00,29.766843,41.802508
2021-12-31 03:00:00,25.647006,64.002897
2021-12-31 06:00:00,22.731885,78.842000
2021-12-31 09:00:00,21.719499,85.092387
2021-12-31 12:00:00,29.548918,53.780205
2021-12-31 15:00:00,40.026148,27.392646
2021-12-31 18:00:00,41.244712,19.955186
2021-12-31 21:00:00,37.645491,22.674331


In [18]:
datos.drop('Unnamed: 0', axis=1, inplace=True) # columna irrelevante, aparente indice

In [20]:


# Filtrar datos a partir de 2019
datos = datos[datos.index.year >= 2019]

# Eliminar filas con valores faltantes
datos.dropna(inplace=True)


<ipython-input-20-6d74a0cc087a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos.dropna(inplace=True)


In [21]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8768 entries, 2019-01-01 00:00:00 to 2021-12-31 21:00:00
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Temperatura  8768 non-null   float64
 1   Humedad      8768 non-null   float64
dtypes: float64(2)
memory usage: 205.5 KB


In [23]:
# Calcular los percentiles 95, 50 (mediana) y 5 para cada día
percentiles_diarios = datos.resample('D')['Temperatura'].agg([lambda x: np.percentile(x, 95),
                                                              lambda x: np.percentile(x, 50),
                                                              lambda x: np.percentile(x, 5)])
percentiles_diarios.columns = ['Percentil_95', 'Mediana', 'Percentil_5']
percentiles_diarios.head()

,Percentil_95,Mediana,Percentil_5
Fecha,,,
2019-01-01,37.100394,29.849186,25.850853
2019-01-02,37.478314,31.976560,28.174603
2019-01-03,37.998732,32.083127,28.178257
2019-01-04,36.794277,32.974816,29.231984
2019-01-05,34.547377,29.600157,24.374377


from matplotlib import pyplot as plt
_df_0['Percentil_95'].plot(kind='hist', bins=20, title='Percentil_95')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Mediana'].plot(kind='hist', bins=20, title='Mediana')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Percentil_5'].plot(kind='hist', bins=20, title='Percentil_5')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='Percentil_95', y='Mediana', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_4.plot(kind='scatter', x='Mediana', y='Percentil_5', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_5['Percentil_95'].plot(kind='line', figsize=(8, 4), title='Percentil_95')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_6['Mediana'].plot(kind='line', figsize=(8, 4), title='Mediana')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_7['Percentil_5'].plot(kind='line', figsize=(8, 4), title='Percentil_5')
plt.gca().spines[['top', 'right']].set_visible(False)

In [24]:
percentiles_diarios.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1096 entries, 2019-01-01 to 2021-12-31
Freq: D
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Percentil_95  1096 non-null   float64
 1   Mediana       1096 non-null   float64
 2   Percentil_5   1096 non-null   float64
dtypes: float64(3)
memory usage: 34.2 KB


In [22]:



# Desplazar para obtener los percentiles del siguiente día como variables objetivo
percentiles_diarios['Percentil_95_siguiente_dia'] = percentiles_diarios['Percentil_95'].shift(-1)
percentiles_diarios['Mediana_siguiente_dia'] = percentiles_diarios['Mediana'].shift(-1)
percentiles_diarios['Percentil_5_siguiente_dia'] = percentiles_diarios['Percentil_5'].shift(-1)

# Eliminar filas con valores faltantes
percentiles_diarios.dropna(inplace=True)

# Seleccionar características (por ejemplo,buscar variables adicionales)
X=percentiles_diarios[['Percentil_95','Mediana','Percentil_5']].values
y = percentiles_diarios[['Percentil_95_siguiente_dia','Mediana_siguiente_dia','Percentil_5_siguiente_dia']].values

# Escalar las características
scaler_X = StandardScaler()
X_escalado = scaler_X.fit_transform(X)

# Dividir datos en conjuntos de entrenamiento y validación
X_entrenamiento, X_validacion, y_entrenamiento, y_validacion = train_test_split(X_escalado, y, test_size=0.2, random_state=SEED, shuffle=False)


# Definir función objetivo para Optuna
def objetivo(trial):
    num_capas = 1
    tf.keras.utils.set_random_seed(SEED)

    num_capas = 1
    num_unidades = trial.suggest_categorical('num_unidades', [16, 32, 64])


    # Definir la arquitectura del modelo
    modelo = Sequential()
    modelo.add(Dense(num_unidades, activation='relu', input_shape=(X_entrenamiento.shape[1],)))
    for _ in range(num_capas - 1):
        modelo.add(Dense(num_unidades, activation='relu'))

    # Capa de salida para predecir los 3 percentiles (95, 50, 5)
    modelo.add(Dense(3, activation='linear'))

    # Compilar el modelo
    modelo.compile(optimizer=Adam(),
                   loss='mse',  # Para regresión, se usa 'mean squared error'
                   metrics=['mae'])  # Mean Absolute Error (MAE) para evaluación

    # Entrenar el modelo
    modelo.fit(X_entrenamiento, y_entrenamiento, validation_data=(X_validacion, y_validacion), epochs=30, batch_size=256, verbose=0, shuffle=False)

    # Evaluar el modelo en el conjunto de validación
    _, val_mae = modelo.evaluate(X_validacion, y_validacion, verbose=0)

    return val_mae

# Realizar la optimización de hiperparámetros usando Optuna
estudio = optuna.create_study(direction='minimize')  # Minimizamos el MAE en lugar de maximizar
estudio.optimize(objetivo, n_trials=10)

# Obtener los mejores hiperparámetros
mejor_num_capas =  1
mejor_num_unidades = estudio.best_params['num_unidades']

print("Mejores Hiperparámetros:")
print("Número de Capas:", 1)
print("Número de Unidades:", mejor_num_unidades)



# Entrenar el modelo final usando los mejores hiperparámetros
modelo_final = Sequential()
modelo_final.add(Dense(mejor_num_unidades, activation='relu', input_shape=(X_entrenamiento.shape[1],)))
for _ in range(mejor_num_capas - 1):
    modelo_final.add(Dense(mejor_num_unidades, activation='relu'))
modelo_final.add(Dense(3, activation='linear'))

modelo_final.compile(optimizer=tf.keras.optimizers.Adam(),
                     loss='mse',
                     metrics=['mae'])

# Entrenar el modelo final
modelo_final.fit(X_entrenamiento, y_entrenamiento, validation_data=(X_validacion, y_validacion), epochs=50, batch_size=256, verbose=1, shuffle=False)

# Evaluar en conjunto de prueba
mae = modelo_final.evaluate(X_validacion, y_validacion, verbose=0)
print(f'MAE en validación: {mae}')


[I 2024-09-11 21:00:43,206] A new study created in memory with name: no-name-7e5b97b1-3847-432e-a389-7222bdd8f6ed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2024-09-11 21:00:47,457] Trial 0 finished with value: 20.4274845123291 and parameters: {'num_unidades': 64}. Best is trial 0 with value: 20.4274845123291.
[I 2024-09-11 21:00:50,564] Trial 1 finished with value: 21.118589401245117 and parameters: {'num_unidades': 32}. Best is trial 0 with value: 20.4274845123291.
[I 2024-09-11 21:00:53,562] Trial 2 finished with value: 21.93014144897461 and parameters: {'num_unidades': 16}. Best is trial 0 with value: 20.4274845123291.
[I 2024-09-11 21:00:57,575] Trial 3 finished with value: 21.93014144897461 and

Mejores Hiperparámetros:
Número de Capas: 1
Número de Unidades: 64
Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 629.9698 - mae: 23.9944 - val_loss: 588.8467 - val_mae: 22.9064
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 627.0723 - mae: 23.9342 - val_loss: 586.0829 - val_mae: 22.8439
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 624.1583 - mae: 23.8735 - val_loss: 583.2867 - val_mae: 22.7806
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 621.2073 - mae: 23.8120 - val_loss: 580.4520 - val_mae: 22.7162
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 618.2095 - mae: 23.7494 - val_loss: 577.5688 - val_mae: 22.6506
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 615.1569 - mae: 23.6855 - val_loss: 574.6271 - val_mae: 22.5834
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 612.0364 - mae: 23.6200 - val_loss: 571.6180 - val_mae: 22.5145
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 608.8347 - mae: 23.5526 - val